In [ ]:
import requests
import json
import os

# Configuration
Please replace the dataset id, file id, API keys and other user specific information to suit your own needs.

In [ ]:
# special instance for MaRDA
clowder_base_uri = "http://141.142.220.44/api"

In [ ]:
base_headers = {'X-API-key': "replace with your own key"}
headers = {**base_headers, 'Content-type': 'application/json', 
           'accept': 'application/json'}

# Create Collections

In [ ]:
def create_collection(clowder_base_uri, name, description='', space_id=''):
    '''
        param: name, description, spaces id 
        note: it's different from dataset, dataset can be added to multiple collections and spaces
        here collection can only choose ONE space
    '''

    payload = json.dumps({'name': name,
                          'description': description,
                          'space': space_id
                         })
    r = requests.post(clowder_base_uri + '/collections',
                     data=payload,
                     headers=headers)
    
    if r.status_code == 200:
        return r.json()
    else:
        return r.text

# Function to create datasets under this collection

In [ ]:
def create_dataset(clowder_base_uri, name, description, access, space=None, collection=None):
    '''
     params: name, description, access: PUBLIC vs PRIVATE, 
         space: a list of string can be empty,
         collection: a list of string, can be empty
    '''

    payload = json.dumps({'name':name, 
                          'description':description,
                          'access':access,
                          'space':space,
                          'collection':collection}) 

    r = requests.post(clowder_base_uri + '/datasets/createempty',
                     data=payload,
                     headers=headers)
    if r.status_code == 200:
        return r.json()
    else:
        return r.text

In [ ]:
def create_folder(clowder_base_uri, name, parentDatasetId, parentId, parentType="dataset"): 
    '''
     parentId could be a folder Id; hence could be different (or the same)than parentDatasetId
    '''

    payload = json.dumps({"name": name, 
                           "parentId": parentDatasetId,
                           "parentType": parentType # or folder
                          }) 

    r = requests.post(clowder_base_uri + '/datasets/'+ parentDatasetId + '/newFolder',
                     data=payload,
                     headers=headers)
    if r.status_code == 200:
        return r.json()
    else:
        return r.text

# Function to list files in a dataset

In [ ]:
def list_files_of_dataset(clowder_base_uri, dataset_id):
    r = requests.get(clowder_base_uri + '/datasets/' + dataset_id + '/listFiles', # list those not infolder
                     headers=headers)
    if r.status_code == 200:
        return r.json()
    else:
        return r.text

# Function to upload files

In [ ]:
def upload_files_to_dataset(clowder_base_uri, dataset_id, folder_id, filenames):
#     headers["Content-type"] = 'multipart/form-data'
    files = [('File', open(fname, 'rb')) for fname in filenames]
    r = requests.post(clowder_base_uri + '/uploadToDataset/' + dataset_id + '?folder_id=' + folder_id,
                      files=files, headers=base_headers)
    if r.status_code == 200:
        return r.json()
    else:
        return r.text

In [ ]:
def move_files_to_folder(clowder_base_uri, dataset_id, folder_id, file_id):
    '''
    Move existing file to a new folder within the same dataset
    '''
    
    payload = json.dumps({"folderId": folder_id}) 

    r = requests.post(clowder_base_uri + '/datasets/'  + dataset_id + '/moveFile/' + folder_id + '/' + file_id,
                      data=payload,
                      headers=base_headers)
    if r.status_code == 200:
        return r.json()
    else:
        return r.text

## Function to Add Metadata

In [ ]:
def add_user_metadata_to_dataset(clowder_base_uri, dataset_id, metadata):
    '''
        metadata can be any key-value pair
        default metadat: Audience, CSDMS Standard Name, Date and Time, Funding Institution, 
            GeoJSON, Grant Number, ODM2 Variable Name, Primary/Initial Publication, Principal Investigator(s),
            References, Related Publications, SAS Spatial Geocode, SAS Variable Name, Time Periods, Unit
    '''
    
    payload = json.dumps(metadata)
    r = requests.post(clowder_base_uri + '/datasets/' + dataset_id +'/usermetadata',
                     data=payload,
                     headers=headers)
    if r.status_code == 200:
        return r.json()
    else:
        return r.text

In [ ]:
def add_user_metadata_to_file(clowder_base_uri, file_id, metadata):
    '''
     metadata can be any key-value pair.
     a few fields that defaulted by clowder: Audience, CSDMS Standard Name, Date and Time, Funding Institution, 
        GeoJSON, Grant Number, ODM2 Variable Name, Primary/Initial Publication, Principal Investigator(s),
        References, Related Publications, SAS Spatial Geocode, SAS Variable Name, Time Periods, Unit
    '''
    
    payload = json.dumps(metadata)
    r = requests.post(clowder_base_uri + '/files/' + file_id +'/usermetadata',
                     data=payload,
                     headers=headers)
    if r.status_code == 200:
        return r.json()
    else:
        return r.text

## loop through the folder structure on host machine and batch publish

In [ ]:
# you can put a collection under a certain space
space_id = "62ebdf11e4b0fd3895ba3788"
collection_name = "Spherical nanoindentation stress-strain curves of primary-α grains in Ti5-2.5, Ti811, Ti64, Ti6242 and Ti6246 \
alloys"
collection_description = "Recently established spherical indentation stress-strain protocols have demonstrated the feasibility \
of measuring reliably the mechanical responses at different material structure length scales in a broad range of \
structural alloys. In the present study, we apply these high-throughput protocols on the primary α-phase grains in\
polycrystalline samples of Ti-5Al-2.5Sn, Ti-8Al-1Mo-1V, Ti-6Al-4V, Ti-6Al-2Sn-4Zr-2Mo and Ti-6Al-2Sn-4Zr-6Mo to \
aggregate a large experimental dataset that documents systematically the effects of α -phase chemical composition \
and grain orientation on the measured values of indentation modulus and the indentation yield strength. This dataset\
is being offered to the materials community in an open repository to allow further analyses of the effect of chemical\
composition of the α -phase on its single crystal elastic and plastic properties. This study clearly establishes the \
feasibility and tremendous value of spherical indentation stress-strain protocols for documenting the grain-scale \
anisotropic mechanical responses of different α -phase compositions in high-throughput assays."

response = create_collection(clowder_base_uri, collection_name, collection_description, space_id)
collection_id = response["id"]
print("Collection " + collection_name + "(" + collection_id + ")" + " created.")

path = "./data/Spherical nanoindentation stress-strain curves of primary-α grains in Ti5-2.5 Ti811 Ti64 Ti6242 \
and Ti6246 alloys/Data Alpha Ti Alloys/"
dataset_names = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

for dataset_name in dataset_names:
    
    # create dataset
    dataset_resp = create_dataset(clowder_base_uri, 
                                  name=dataset_name, 
                                  description=dataset_name, 
                                  access="PUBLIC", # "PUBLIC or PRIVATE"
                                  space=[space_id], # put it in a space
                                  collection=[collection_id]) # collection id we previously created
    
    dataset_id = dataset_resp["id"]
    print("\n=========")
    print("dataset " + dataset_name + "(" + dataset_id + ")" + " created.")
    
    # create folder
    folder_names = [f for f in os.listdir(os.path.join(path, dataset_name))
                                          if os.path.isdir(os.path.join(path, dataset_name, f))]
    for folder_name in folder_names:
        folder_resp = create_folder(clowder_base_uri, 
                                      folder_name, 
                                      parentDatasetId=dataset_id, 
                                      parentId=dataset_id, 
                                      parentType="dataset")
        folder_id = folder_resp["id"]
        print("folder " + folder_name + "(" + folder_id + ")" + " created.")
        
        # upload files
        file_fullpaths = [os.path.join(path, dataset_name, folder_name, file)
                          for file in os.listdir(os.path.join(path, dataset_name, folder_name)) 
                          if os.path.isfile(os.path.join(path, dataset_name, folder_name, file))]
        file_resp = upload_files_to_dataset(clowder_base_uri, dataset_id, folder_id, file_fullpaths)
        for f in file_resp["ids"]:
            print("file " + f["name"] + "(" + f["id"] + ")" + " uploaded.")